## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-19-18-54-44 +0000,nyt,Relief in Ukraine as Zelensky Avoids Disaster ...,https://www.nytimes.com/2025/08/19/world/europ...
1,2025-08-19-18-54-00 +0000,wsj,Housing construction in the U.S. jumped much m...,https://www.wsj.com/economy/housing/housing-st...
2,2025-08-19-18-50-17 +0000,wapo,Europeans hail Trump’s promise of securing Ukr...,https://www.washingtonpost.com/world/2025/08/1...
3,2025-08-19-18-49-37 +0000,nyt,"Jules Witcover, Political Reporter and Columni...",https://www.nytimes.com/2025/08/18/business/me...
4,2025-08-19-18-49-13 +0000,bbc,Netanyahu accuses Australian PM of 'betraying'...,https://www.bbc.com/news/articles/c5ylkpp77qxo...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2305/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
21,trump,47
1,ukraine,26
212,canada,16
2,zelensky,16
99,new,15


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
205,2025-08-19-02-43-00 +0000,wsj,Trump Pushes for Putin and Zelensky Meeting in...,https://www.wsj.com/world/trump-pushes-for-pea...,125
118,2025-08-19-14-07-00 +0000,wsj,"After months of effort, President Trump says a...",https://www.wsj.com/world/trump-plans-to-get-p...,118
152,2025-08-19-10-25-29 +0000,nypost,Putin and Zelensky could meet within next two ...,https://nypost.com/2025/08/19/world-news/putin...,104
16,2025-08-19-18-30-03 +0000,nypost,Russia open to bilateral or trilateral talks w...,https://nypost.com/2025/08/19/us-news/russia-o...,102
220,2025-08-19-00-41-09 +0000,nyt,Can Zelensky Trust Trump? Ukraine’s Fate May D...,https://www.nytimes.com/2025/08/18/us/politics...,98


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
205,125,2025-08-19-02-43-00 +0000,wsj,Trump Pushes for Putin and Zelensky Meeting in...,https://www.wsj.com/world/trump-pushes-for-pea...
39,82,2025-08-19-17-52-00 +0000,wsj,"Flight Attendants Win Boarding Pay, Four-Year ...",https://www.wsj.com/business/airlines/air-cana...
278,56,2025-08-18-19-54-53 +0000,nypost,Putin claims Ukraine should give up Donetsk be...,https://nypost.com/2025/08/18/world-news/putin...
101,51,2025-08-19-14-54-58 +0000,nyt,Netanyahu Faces Pressure From Far Right Over N...,https://www.nytimes.com/2025/08/19/world/middl...
227,47,2025-08-19-00-01-00 +0000,wsj,The Trump administration has restored a websit...,https://www.wsj.com/politics/policy/lawmakers-...
71,33,2025-08-19-16-34-37 +0000,nypost,First biography from Oct. 7 hostage Eli Sharab...,https://nypost.com/2025/08/19/world-news/first...
1,31,2025-08-19-18-54-00 +0000,wsj,Housing construction in the U.S. jumped much m...,https://www.wsj.com/economy/housing/housing-st...
58,28,2025-08-19-17-09-45 +0000,nypost,China’s Pop Mart sees profit soar 400% as Labu...,https://nypost.com/2025/08/19/business/chinas-...
236,27,2025-08-18-23-13-00 +0000,wsj,Texas Democrats ended a two-week walkout over ...,https://www.wsj.com/politics/policy/texas-demo...
245,27,2025-08-18-22-20-19 +0000,nypost,MSNBC gets mercilessly mocked over new MS NOW ...,https://nypost.com/2025/08/18/media/msnbc-gets...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
